In [1]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics, preprocessing

import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')
%matplotlib inline

C:\Users\andrz\AppData\Local\Temp\ipykernel_12304\3527984445.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-ticks')


In [2]:
df_o = pd.read_csv("data/model_data.csv")
print(df_o.shape)
df_o.head()

(2178, 12)


,ID,salary_perc,age,games_played_perc,games_started_perc,minutes_played,avg_minutes_played,WS48,PER,team_successes,player_successes,WS
0,webbech01,0.300,27,0.866667,0.866667,3184,40.821,0.186,26.492,1,2,12.338000
1,smithjo02,0.107,25,0.841463,0.719512,1941,28.130,0.109,14.600,0,0,4.407687
2,olajuha01,0.119,38,0.707317,0.670732,1545,26.638,0.155,20.700,0,0,4.989063
3,robinda01,0.224,35,0.978947,0.978947,2780,29.892,0.245,27.926,2,2,14.189583
4,mutomdi01,0.337,34,0.933333,0.933333,3572,36.449,0.150,24.959,3,5,11.162500


In [3]:
df = df_o.copy()
len(df)

2178

In [4]:
df = df.dropna()
len(df)

2175

In [5]:
target = "salary_perc"
features = ['age', 'games_played_perc', 'games_started_perc', 'minutes_played', 'avg_minutes_played', 'WS48', 'PER', 'team_successes', 'player_successes']

In [6]:
for column in df.columns.values:
    check_nan = df[f'{column}'].isnull().values.any()
    print(f"{column}: {check_nan}")

ID: False
salary_perc: False
age: False
games_played_perc: False
games_started_perc: False
minutes_played: False
avg_minutes_played: False
WS48: False
PER: False
team_successes: False
player_successes: False
WS: False


In [7]:
for index, row in df.iterrows():
    check_nan = row.isnull().values.any()
    if check_nan:
        print(row)

In [12]:
df = df.drop(columns="ID")

In [21]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pathlib import Path
import pandas as pd


def TestRFRegress(
    nFolds=5,
    randomState=2020,
    debug=False,
    features=None,
    target_col: str = "target",
    *args,
    **kwargs
):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    testResults = []
    trainResults = []
    predictions = []
    indices = []

    if not features:
        features = [col for col in df.columns if col != target_col]

    for train, test in kf.split(df.index.values):
        reg = RandomForestRegressor(
            *args, **kwargs, random_state=randomState, n_jobs=-1
        )
        if debug:
            print(reg)

        reg.fit(df.iloc[train][features], df.iloc[train][target_col])

        predsTrain = reg.predict(df.iloc[train][features])
        preds = reg.predict(df.iloc[test][features])

        predictions.append(preds.tolist().copy())
        indices.append(df.iloc[test].index.tolist().copy())

        trainScore = mean_absolute_error(df[target_col].iloc[train], predsTrain)
        testScore = mean_absolute_error(df[target_col].iloc[test], preds)

        trainResults.append(trainScore)
        testResults.append(testScore)

        if debug:
            print("Train MAE:", trainScore, "Valid MAE:", testScore)

    return trainResults, testResults, predictions, indices

In [22]:
trainResults, testResults, predictions, indices = TestRFRegress(debug=True, target_col="salary_perc")

RandomForestRegressor(n_jobs=-1, random_state=2020)
Train MAE: 0.011133816091954017 Valid MAE: 0.031471494252873565
RandomForestRegressor(n_jobs=-1, random_state=2020)
Train MAE: 0.011316942528735632 Valid MAE: 0.0314420459770115
RandomForestRegressor(n_jobs=-1, random_state=2020)
Train MAE: 0.01132791954022988 Valid MAE: 0.03065636781609195
RandomForestRegressor(n_jobs=-1, random_state=2020)
Train MAE: 0.011849798850574706 Valid MAE: 0.028411402298850575
RandomForestRegressor(n_jobs=-1, random_state=2020)
Train MAE: 0.011374252873563213 Valid MAE: 0.03133239080459771


In [ ]:
# Pętla po parametrze n_estimators
for k in [10, 25, 50, 100, 200, 500, 1000]:
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=k)
    print(k, np.mean(trainResults), np.mean(testResults), np.mean(trainResults) - np.mean(testResults))

In [ ]:
for k in range(2,22,2):
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100, max_depth=k)
    print(k, np.mean(trainResults), np.mean(testResults), np.mean(trainResults) - np.mean(testResults))

Jak widać w przypadku tego zbioru algorytm skutecznie korzysta nawet z bardzo głębkoich drzew. Z drugiej strony, można zaobserwować jak wraz ze zwiększaniem głębokości drzew zwiększa się przetrenowanie modelu. Przyjrzyjmy się kolejnemu hiperparametrowi - liczby zmiennych w pojedynczym drzewie. O ile nie ustalimy inaczej, algorytm automatycznie ograniczy liczbę zmiennych na dane drzewo ustalać liczbę jako ```sqrt(n_features)```. Zobaczmy czy inna wartość będzie lepsza.

In [ ]:
# lizczba zmiennych objaśniających
print(len(features))

In [ ]:
for k in range(2, len(features) + 1,2):
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100, max_depth=10,
                                                                    max_features=k)
    print(k, np.mean(trainResults), np.mean(testResults), np.mean(trainResults) - np.mean(testResults))

Przypadkiem okazało się, że reguła kciuka (4 zmienne) to rzeczywiście najlepsza wartość. Trzeba jednak pamiętać, że nie zawsze tak będzie! Testowanie hiperparametrów to zawsze bardzo ważny krok w przypadku zaawansowanych algorytmów machine learningu.

W dwóch kolejnych pętlach sprawdzimy działanie `min_samples_split` oraz `min_samples_leaf`.

In [ ]:
for k in [2, 4, 6, 8, 10, 15, 20, 30]:
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100, max_depth=10,
                                                                    max_features=4,
                                                                    min_samples_split=k)
    print(k, np.mean(testResults))

Skoro musimy mieć co najmniej 4 do splitu to po splicie możemy już przetestować wyłącznie wartości 1 i 2.

In [ ]:
for k in [1, 2, 4]:
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100, max_depth=10,
                                                                    max_features=4,
                                                                    min_samples_split=6, min_samples_leaf=k)
    print(k, np.mean(testResults))

Wydaje się, że najlepszy wynik osięgnęliśmy przy głębokości 20, maksymalnej liczbie featerów 4, obserwacji przed splitem 4 i min w liściu po splicie 1. Dla tych parametrów (ale większej liczby drzew) zapiszmy predykcje naszego mdoelu i porównajmy z wczesniejszymi wynikami.

In [ ]:
trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=500, max_depth=14,
                                                                    max_features=6,
                                                                    min_samples_split=4, min_samples_leaf=1)
print(k, np.mean(testResults))

modelRF = {
    "name":"RF",
    "description":"Model RF, ze zmiennymi kategorycznymi z LE",
    "specification":'n_estimators=500, max_depth=20, max_features=4, min_samples_split=4, min_samples_leaf=1',
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
}

In [ ]:
import pickle

# Otwieramy plik do zapisu binarnego z wykorzystenim with
with open("model_RF_1.p", "wb") as fp:
    # Zapisujemy obiekt do wskaźnika pliku
    pickle.dump(modelRF, fp)

In [ ]:
# Wczytajmy też wcześniej zapisane modele
with open("model_ekonometria_1.p", "rb") as fp:
    modelEkonometria = pickle.load(fp)
    
with open("model_svm_1.p", "rb") as fp:
    modelSVM = pickle.load(fp)
    
with open("model_kNN_1.p", "rb") as fp:
    modelkNN = pickle.load(fp)

### Porównanie wyników ze wcześniejszymi modelami
Niektórzy już pewnie zauważyli, że wielokrotne kopiowanie kodu funkcji (np tej do rysowania krzywej ROC) pomiędzy notatnikami jest niepraktyczne i kłóci się z zasadą DRY (Don't Repeat Yourself). Zobaczmy teraz jak łatwe jest importowanie w Pythonie bibliotek i funkcji tak żeby korzystać z nich wielokrotnie.

Spójrzmy na zawartośc pliku helpers.py w naszym głównym katalogu. Skopiowaliśmy tam po prostu zawartośc naszej funkcji. Dzięki temu wystarczy w tym notatniku umieścić poniższą linię, aby funkcja była już dla nas dostępna.

In [ ]:
from helpers import plotROCs

In [ ]:
res2plot = []
true = (df[target]=="yes").astype(int).sort_index()
for result in [modelRF, modelkNN, modelEkonometria, modelSVM]:
    # Tworzymy wektor predykcji tworząc serię pandasa, a następnie sortujemy indeks
    pred = pd.Series(sum(result["predictions"], []), index=sum(result["indices"], [])).sort_index()
    res2plot.append((true, pred, result["name"]))
    
# Wywołujemy funkcję do rysowania
plotROCs(res2plot)

Jak widać, dzięki lasowi losowemu uzyskano najlepszy jak dotąd wynik. Z pozostałych parametrów klasyfikatora `RandomForest` warto omówić jeszcze następujące argumenty:
* `bootstrap` - domyślnie True, jeżeli wybrane zostanie False, wyłączymy bagging, a każde drzewo będzie budowane na pełnym zbiorze danych
* `min_weight_fraction_leaf` - float, który określa jak mała może być część w liściu po splicie (od 0 do 1)
* `min_impurity_decrease` - wartość float między 0 a 1, która mówi jak bardzo musi się poprawić klasyfikacja po splicie aby można było dokonać splitu.

## 3.4 Szukanie optymalnych wartości hiperparametrów
Random forest jest pierwszy algorytmem, gdzie liczba hiperparametrów robi się na tyle zauważalna, że warto coś więcej powiedzieć o szukaniu hiperparametrów. Dwie najczęściej stosowane metody to *grid search* oraz *random search*. W pierwszym przypadku ustalamy sztywną siatkę wartości dla wszystkich analizowanych zmiennych, a w drugim, dla każdej zmiennej losujemy wartości.

Istnieje wiele bibliotek, które specjalizują się w szukaniu hiperparametrów. Zanim po nie sięgniemy warto by jednak było najpierw zobaczyć jak prosta jest samodzielna implementacja random search i jakie daje efekty.

In [ ]:
# Przygotujmy listę do zapisywnia wyników
results = []

In [ ]:
import random
# Teraz napiszmy pętle, w której będziemy iterować kolejne wyszukiwania.
for k in range(50):
    # Przygotujmy słownik parametrów
    # Jako punkt odniesienia do wybranych przedziałów posłużą nam wcześniejsze wyniki
    params = {
        "max_depth" : random.randint(6, 22),
        "max_features" : random.randint(2, 12),
        "min_samples_split" : random.randint(2, 9),
    }
    # Wartość min_samples_leaf musi być mniejsza niż min_samples_split
    params["min_samples_leaf"] = random.randint(1, params["min_samples_split"])
    
    # Estymacja modelu dla wylosowanych hiperparametrów
    trainResults, testResults, predictions, indices = CVTestRFClass(n_estimators=100,
                                                                    max_depth=params["max_depth"],
                                                                    max_features=params["max_features"],
                                                                    min_samples_split=params["min_samples_split"],
                                                                    min_samples_leaf=params["min_samples_leaf"])
    
    # Zapiszmy wynik
    results.append((np.mean(testResults), params.copy()))
    
    # Wyświetlmy wynik bieżącej iteracji
    print(params, np.mean(testResults))
    
    # Co 10 iteracji wyświetlmy sobie nasze dotychczasowe top 5
    if k>1 and k%10==0:    
        print("\n Top 5:")
        for score, params in sorted(results, key=lambda x: x[0], reverse=True)[0:5]:
            print(params, score)
        print()

In [ ]:
# Zobaczmy jak wyglądają wyniki od najlepszego graficznie
plt.plot([x for x, y in sorted(results, key=lambda x: x[0], reverse=True)])

Jak widać zdecydowana większość (60-80%) jest na relatywnie wysokim poziomie, a tylko około 20% to kiepskie wyniki. Co więcej medianowy wynik nie różni się bardzo mocno od najlepszego z wyników.

In [ ]:
# Możemy tez zobaczyć jak z czasem poprawiał się najlepszy wynik:
bestHistory = []
for k in range(1,len(results)):
    # List comprehension: dla k pierwszych wartości zwróć maximum
    bestHistory.append(np.max([x for x, y in sorted(results[:k], key=lambda x: x[0], reverse=True)]))
# Wykres historii
plt.plot(bestHistory)

Powyższy wykres wyraźnie pokazuje, że już po 15 iteracjach *random search* znaleźliśmy hiperparametry, który były bardzo blisko optymalnej kombinacji znalezionej w procedurze.

## 3.5. Feature importance
W zależności od algorytmu, że obiekt zwracany przez funkcję estymującą model, ma wbudowane metody, które pomagają opracować wyniki analiz. Może to być np. zapisywanie modelu lub robienie ponownej predykcji na innych danych. W przypadku Random Forest taką dodatkową metodą jest np. *feature importance*. W związku z tym jako wynik walidacji krzyżowej warto jest przekazać również utworzone  modele. Zróbmy jeszcze jedną modyfikację funkcji walidacji krzyżowej - dodajmy parametr `saveModels = False`.

In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


# Przygotujmy wrapper
def CVTestRFClass(nFolds = 5, randomState=2020, debug=False, features=features, saveModels = False, *args, **kwargs):
    
    # Przygotujmy walidację krzyżową
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Aby oszczędzać pamięć informacja o foldach to wyłącznie numery wierszy
    testResults = []
    
    # Dla kompletności dodamy informację o w ynikach na zbiorze treningowym
    trainResults = []
    
    # Przechowajmy również predykcje dla poszczególnych foldów
    predictions = []
    # Razem z informacją o tym, jaki był ich indeks w oryginalnym zbiorze danych
    indices = []
    
    # Przygotujmy liste modeli do zachowania
    models = []
    
    # Pętla walidująca modele
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = RandomForestClassifier(*args, **kwargs, random_state=randomState, n_jobs=-1)
        if debug:
            print(clf)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])

        # Przygotowanie prognoz dla zbioru treningowego i testowego
        # UWAGA sklearn zwracał będzie dwie kolumny prawdopodobieństw dla obydwu klas
        predsTrain = clf.predict_proba(df.iloc[train][features])[:,1]
        preds = clf.predict_proba(df.iloc[test][features])[:,1]
        
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Policzenie dopasowania metryką ROC-AUC
        trainScore = roc_auc_score((df[target].iloc[train]=="yes").astype(int), predsTrain)
        testScore = roc_auc_score((df[target].iloc[test]=="yes").astype(int), preds)
        
        # zapisanie wyników z iteracji
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
        # Zapisanie wyników do listy models
        if saveModels:
            models.append(clf)
    
    # Lista zwracanych wyników - zwróćmy uwagę na nowy obiekt models
    return trainResults, testResults, predictions, indices, models

Wykonajmy trening modelu:

In [ ]:
trainResults, testResults, predictions, indices, models = CVTestRFClass(n_estimators=500, max_depth=20,
                                                                    max_features=4,
                                                                    min_samples_split=4, min_samples_leaf=1,
                                                                        saveModels=True)

Dzięki zapisaniu modeli, możemy wybrać jeden z nich i zobaczyć które zmienne były najczęściej wykorzystywane w drzewach, które algorytm może uznać za najważniejsze. Zobaczmy czy wynik będzie podobny pomiędzy foldami.

In [ ]:
imps = list(zip(models[0].feature_importances_, features))
imps.sort(reverse=True)
imps

In [ ]:
imps = list(zip(models[1].feature_importances_, features))
imps.sort(reverse=True)
imps

Jak widać uzyskujemy dosyć podobne wnioski jak w przypadku analizy z uzyciem algorytmu kNN. Duration hest zdecydowanie najważniejszą zmienną. Zmienne, age, day i balance również okazywały się być istotne w obydwu przypadkach.

### Ćwiczenia
* Sprawdź czy zmiana parametrów min_weight_fraction_leaf oraz min_impurity_decrease może pomóc nam poprawić nasz model.
* Na zbiorze Boston Housing przygotuj model regresyjny z wykorzystaniem Random Forest Regressor.